In [19]:
from csv import reader
import csv
from csv import DictReader
import re
import pandas as pd
from fuzzywuzzy import fuzz


file = "Baghizadhe2019"
progress = 0

def fuzzy_classification_title (title_compare, author, year, reference_compare):
    if fuzz.partial_ratio(title_compare, reference_compare) < 87: 
        return False
    else:
        return fuzzy_classification_author_year(author, year, reference_compare)

def fuzzy_classification_author_year (author, year, reference_compare):
    first_author = author.split(",")[0]
    first_author = alphanumeric_filter_lower(first_author)
    if first_author not in reference_compare:
        if fuzz.partial_ratio(first_author, reference_compare) < 70:
            if author == "[No author name available]":
                if year_child not in reference_compare:
                    return False
            else:
                return False
    return True

def alphanumeric_filter_lower(string):
    alphanumeric_filter = filter(str.isalnum, string)
    string = "".join(alphanumeric_filter)
    string = string.lower()
    return string
     

    
#creating nodelist csv
with open("./papers/" + file + "/" + file + "_joined" + '.csv', 'w') as joined_csv:
    writer = csv.writer(joined_csv)
    writer.writerow(["Authors", "Title", "Year", "Source Title", "Document Type", "FAMILY_TYPE"])
    #iterating through parent and child documents
    with open("./papers/" + file + "/scopus_parents.csv", "r") as read_obj_parents:
        csv_dict_reader_parents = DictReader(read_obj_parents)
        with open("./papers/" + file + "/scopus_children.csv", "r") as read_obj_children:
            csv_dict_reader_children = DictReader(read_obj_children)
            
            # parent row
            for row_parent in csv_dict_reader_parents:
                progress += 1
                print(progress)
                
                author_parent = row_parent['\ufeffAuthors']
                title_parent = row_parent['Title']
                year_parent = row_parent['Year']
                source_title_parent = row_parent['Source title']
                document_type_parent = row_parent['Document Type']
                references_parent = row_parent['References']
                already_appended_titles = []
                
                reference_counter = 0
                if references_parent == "":
                    print("ERROR: " + author_parent + ", " + title_parent + " - has no references")
                    continue
                
                writer.writerow([author_parent, title_parent, year_parent, source_title_parent, document_type_parent, "PARENT"])

                for reference in references_parent.split(";"):
                    read_obj_children.seek(0)
                    reference_compare = alphanumeric_filter_lower(reference) 
                    for row_child in csv_dict_reader_children:
                        title_child = row_child["Title"]
                        author_child = row_child['\ufeffAuthors']
                        year_child = row_child['Year']
                        if title_child != "[No title available]":
                            title_compare = alphanumeric_filter_lower(title_child)
                            if title_compare in reference_compare:
                                if len(title_child.split()) < 5:
                                    if fuzzy_classification_author_year (author_child, year_child, reference_compare) == False:
                                        continue
                            else: 
                                continue
                            #else:
                            #    if fuzzy_classification_title(title_compare, author_child, year_child, reference_compare) == False:
                            #        continue
                            #    else:
                            #        print("ITS HAPPENING " + title_child + author_child + year_child + " - " + reference)
                            if title_compare not in already_appended_titles:
                                reference_counter += 1
                                already_appended_titles.append(title_compare)
                                source_title_child = row_child['Source title']
                                document_type_child = row_child['Document Type']
                                writer.writerow([author_child, title_child, year_child, source_title_child, document_type_child, "CHILD"])
                                break
                                
                writer.writerow(["Reference Count: " + str(reference_counter), "", "", "", "", ""])
                writer.writerow(["", "", "", "", "", ""])
    
    read_obj_parents.close()
    read_obj_children.close()
    
    
    # default added articles
    if False:
        pattern_title = '(?<=<t>)(.*?)(?=</t>)'
        pattern_year = '\d{4}'
        reference_counter = 0

        def get_title(line):
            title_match = ""
            if '<t>' in line:
                if '</t>' in line:
                    title_match = re.search(pattern_title, line)
            if title_match != "":
                return title_match.group(0)
            else:
                return title_match


        with open("./papers/" + file + "/" + file, "r") as read_obj_default:
            for line in read_obj_default:
                if '<PARENTARTICLE>' in line:  
                    if reference_counter > 0:
                        writer.writerow(["Reference Count: " + str(reference_counter), "", "", "", "", ""])
                        writer.writerow(["", "", "", "", "", ""])
                if '<t>' in line:
                    title = get_title(line)
                    if title != "":
                        year = re.search(pattern_year, line)
                        if '<PARENTARTICLE>' in line:                   
                            reference_counter = 0
                            writer.writerow(["", title, year, line, "", "", "PARENT"])
                        else: 
                            reference_counter += 1
                            writer.writerow(["", title, year, line, "", "", "CHILD"])

    #read_obj_default.close()
joined_csv.close()

        
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
ERROR: Mähring M., Keil M., Information technology project escalation: A process model - has no references
40
41
42
ERROR: Bartis E., Mitev N., A multiple narrative approach to information systems failure: A successful system that failed - has no references
43
ERROR: Kasi V., Keil M., Mathiassen L., Pedersen K., The post mortem paradox: A Delphi study of IT specialist perceptions - has no references
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
ERROR: Keil Mark, Montealegre Ramiro, Cutting your losses: Extricating your organization when a big project goes awry - has no references
71
72
73
74
75
ERROR: Keil Mark, Mann Joan, Understanding the nature and extent of IS project escalation: Results from a survey of IS audit and control professionals - has no references
76
77
78
79
80
81


In [5]:
test1 = "XDDDDDDDDDDDDtesttesthahahahSENFH.wriquoqiwjeöwerjwXXXD"
test2 = "SENFH."
print(fuzz.partial_ratio(test2, test1))
print(fuzz.token_set_ratio(test2, test1))
<NOSCOPUSREFERENCES> ---



100
17
